In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
GOOGLE_SEARCH_API_KEY=os.getenv("GOOGLE_SEARCH_API_KEY")
GOOGLE_CSE_ID=os.getenv("GOOGLE_CSE_ID")
LANGSMITH_API_KEY=os.getenv("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true" #enable tracing
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [ ]:
#loading local model
MODEL="deepseek-r1:1.5b"
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
llm = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = llm | parser 

In [ ]:
print(chain.invoke("YO, tell me a joke"))

### step 1. Make a vector store ###

In [ ]:
#making a retreiver with vector store out of a pdf document
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path="thesis.pdf")
pages = loader.load_and_split()
print(f"Loaded pdf with {len(pages)} pages")

#### Option A : Simple DocArrayInMemorySearch ####

In [ ]:
# from langchain_community.vectorstores import DocArrayInMemorySearch

# vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)
# retriever = vectorstore.as_retriever()

#### Option B : Facebook AI Similarity Search (FAISS) ####

In [ ]:
#ONLY do this once
from langchain_community.vectorstores import FAISS

#vectorstore = FAISS.from_documents(documents = pages, embedding = embeddings)
#vectorstore.save_local(folder_path='data/',index_name="thesis-index")

In [ ]:
vectorstore = FAISS.load_local(folder_path='data/',
                               index_name='thesis-index',
                               embeddings=embeddings,
                               allow_dangerous_deserialization = True
                               )
retriever = vectorstore.as_retriever()

In [ ]:
retriever.invoke("reinforcement learning")

In [ ]:
#from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

template = '''Answer the question based on the given context. Incase, you 
cannot answer the question based SOLELY on the context, reply "I dont know".

Context: {context}
Question: {question}

'''
prompt = PromptTemplate(template=template, input_variables=['context','question'])

print(prompt.format(
    context="Hi I am susan.",
    question="What is my name?"
))

In [ ]:
chain = prompt | llm | parser
print(chain.invoke({
    "context":"Hi I am susan.",
    "question": "Was ist meine name?"
}))

### step 2. Grading top retrieved document ###

In [ ]:
# LLM for Grading: grades the top document fetched by the retreiver 
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

system_msg = SystemMessagePromptTemplate.from_template(
    "You are an assistant that evaluates documents for relevance and outputs JSON."
)

# Human message is the dynamic input
human_msg = HumanMessagePromptTemplate.from_template("""Give a binary 'yes' or 'no' to grade whether the document is relevant to the question. \n
    Provide the single binary grade as a JSON with a single key 'grade' and no premable or explanation.
    Document: {document}
    Question: {question}
"""
)
chat_prompt = ChatPromptTemplate.from_messages([system_msg, human_msg])

#dummy run
inputs = {'document': [''], 'question':'This is my question'}
formatted_messages = chat_prompt.format_messages(**inputs)
print('Prompt to retrieval grader')
for msg in formatted_messages:
    print(f"{msg.type} : {msg.content}")

In [ ]:
retrieval_grader = chat_prompt | llm | JsonOutputParser()
#retrieval_grader = chat_prompt | llm 

In [ ]:
#dummy run
question = "What is the motivation behind this project?"
docs = retriever.invoke(question)
doc_text = docs[0].page_content

grade = retrieval_grader.invoke({"question": question, "document": doc_text})


#### Sidebar: processing llm output before parsing for robustness ####

In [ ]:
# import regex as re
# def fetch_json(text):
#     match=re.search(r"\{.*\}", text, re.DOTALL)
#     if match:
#         return match.group(0)
#     return None

In [ ]:
# raw_text=retrieval_grader.invoke(inputs)
# clean_text=fetch_json(raw_text)
# json_parser = JsonOutputParser()
# if clean_text is not None:
#     grade=json_parser.parse(clean_text)
# print(grade)

In [ ]:
# print(raw_text)

#### END sidebar: processing llm output before parsing for robustness ####

In [ ]:
grade['grade']

In [ ]:
print(f"Retreiver fetched document: {doc_text}")

### step 3: Generate RAG response chain    ###

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

system_msg = SystemMessagePromptTemplate.from_template(
    "You are an assistant for question-answering tasks."
)

# Human message is the dynamic input
human_msg = HumanMessagePromptTemplate.from_template("""Use the following context to answer the question in three sentences max. If you don't know the answer, just say so. 
Limit your response to a maximum of three sentences.
    Question: {question}
    Context: {context}                                                 
"""
)
chat_prompt = ChatPromptTemplate.from_messages([system_msg, human_msg])

rag_chain = prompt | llm | StrOutputParser()


In [ ]:
#dummy run
docs = retriever.invoke(question)
content_docs = " ".join([doc.page_content for doc in docs])
generated_response = rag_chain.invoke({"question": question, "context": content_docs})

In [ ]:
print(generated_response)

In [ ]:
content_docs

### step 4: Check for Hallucinations ###

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

system_msg = SystemMessagePromptTemplate.from_template(
    "You are a grader assessing whether an answer is supported by a set of facts and outputs JSON."
)

# Human message is the dynamic input
human_msg = HumanMessagePromptTemplate.from_template("""Give a binary 'yes' or 'no' to grade whether the answer is supported by the given set of facts.
    Provide the single grade as a JSON with a single key 'grade' and no premable or explanation.
    
    Answer: {generation}
    Documents: {documents}
""")

chat_prompt = ChatPromptTemplate.from_messages([system_msg, human_msg])
hallucination_grader = chat_prompt | llm | JsonOutputParser()

In [ ]:
print(chat_prompt)

In [ ]:
#dummy test run
grade = hallucination_grader.invoke({"generation": generated_response, "documents": content_docs})

In [ ]:
grade['grade']

In [ ]:

sys_msg = SystemMessagePromptTemplate.from_template("You are a grader assessing whether an answer resolves a question.")

human_msg = HumanMessagePromptTemplate.from_template("""Give a binary grade 'yes' or 'no' to indicate whether the answer is 
    useful to resolve a question. Provide the grade as a JSON with a single key 'grade' and no preamble or explanation.
    
    Question: {question}
    Answer: {generation}
    """)

chat_prompt = ChatPromptTemplate.from_messages([system_msg, human_msg])

answer_grader = chat_prompt | llm | JsonOutputParser()


In [ ]:
#dummy test run
answer_grader=answer_grader.invoke({"question": question, "generation": generated_response})

In [ ]:
print(grade)

### step 5: set up Google search API ###

In [ ]:
from langchain.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool
# from pydantic import BaseModel, Field, root_validator

search = GoogleSearchAPIWrapper(k=3,
                                google_api_key=GOOGLE_SEARCH_API_KEY,
                                google_cse_id=GOOGLE_CSE_ID)
def search_results(query):
    return search.results(query, num_results=3)

web_search_tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search_results
)

In [ ]:
docs=web_search_tool.invoke({'query':'what is Diwali in 2025?'})

In [ ]:
#dummy run
question = "What is the motivation behind this project?"
results=web_search_tool.invoke({'query':question})
results

#### neccesarry declarations before langgraph ####

In [ ]:
from typing_extensions import TypedDict
from typing import List
from langchain_core.documents import Document

In [ ]:
#state
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes: 
    question: string of the question
    generation: LLM generation 
    web_search: whether to add search 
    documents: list of retreived documents from vector store
    """

    question: str
    generation: str
    web_search: str
    documents: List[str]

#node
def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args: 
        state (dict) : Current graph state

    Returns: 
        state (dict): New key added to state: "documents". This key contains retrieved documents
    """

    print("----RETRIEVE----")
    question = state['question']

    #retrieval
    documents = retriever.invoke(question)
    return {'documents':documents, 'question':question}

def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state: "generation". This key contains LLM generation
    """
    print("----GENERATE----")

    question=state['question']
    documents=state['documents']

    #RAG generation 
    generation = rag_chain.invoke({'context':documents,'question':question})
    return {'documents':documents,
            'question':question,
            'generation':generation}

def grade_documents(state):
    
    """
    Determines whether the retrieved documents are relevant to the question
    If any document is not relevant, we set a flag to run web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    filtered_docs=[]
    web_search='No'

    for d in documents:
        answer = retrieval_grader.invoke(
            {'question':question,'document':d.page_content}
        )

        grade = answer['grade']
        
        #check if document is relevant
        if grade.lower() == 'yes':
            print("--GRADE: DOCUMENT RELEVANT--")
            filtered_docs.append(d)
        else:
            print("--GRADE: DOCUMENT NOT!!!! RELEVANT--")
            web_search='Yes'
            continue
    
    return {'documents':filtered_docs,
            'question':question,
            'web_search':web_search}

def web_search(state):
    """
    Web search based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """

    print("----WEB SEARCH----")
    question = state["question"]
    documents = state["documents"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["snippet"] for d in docs])

    web_results = Document(page_content=web_results)
    if documents is not None:
        documents.append(web_results)
    else:
        documents = [web_results]
    return {"documents": documents, "question": question}

    
def decide_to_generate(state):
    """
    Determines whether to generate an answer, or add web search

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    question = state["question"]
    web_search = state["web_search"]
    filtered_documents = state["documents"]

    if web_search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---"
        )
        return "websearch"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    response = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = response["grade"]

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score["score"]
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RETRY---")
        return "not supported"  

### step 6 : Build a graph using langgraph ###

In [ ]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("websearch", web_search)  # web search
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generate



In [ ]:
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve","grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "websearch" : "websearch",
        "generate" : "generate"
    },
)
workflow.add_edge("websearch","generate") #????
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported" : "generate",
        "useful" : END,
        "not useful" : "websearch",
    },
)

In [ ]:
# Compile
app = workflow.compile()

# Test
from pprint import pprint

inputs = {"question": "What is motivation behind this project?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])

#### installations ####

In [ ]:
#%pip install faiss-cpu